In [1]:
import simpy

In [2]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)
        

In [3]:
env = simpy.Environment()
car = Car(env)
env.run(until=30)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14
Start driving at 19
Start parking and charging at 21
Start driving at 26
Start parking and charging at 28


In [2]:
import simpy

env = simpy.Environment()

# função para realizar o tempor de internação de um paciente
def paciente(env, nome, tempo_internacao):
    print(f'Entrada na UTI em {env.now} de {nome}' )
    yield env.timeout(tempo_internacao)
    print(f'Saída da UTI em {env.now} de {nome}')
    
# função para criar pacientes
def cria_paciente(env):
    paciente_nome = 0
    intervalo_criacao = 2
    tempo_internacao = 10
    while True:
        yield env.timeout(intervalo_criacao)
        
        print(f'Paciente {paciente_nome} criado em {env.now}')
        env.process(paciente(env, paciente_nome, tempo_internacao))
        paciente_nome += 1

env.process(cria_paciente(env))
env.run(until=30)


Paciente 0 criado em 2
Entrada na UTI em 2 de 0
Paciente 1 criado em 4
Entrada na UTI em 4 de 1
Paciente 2 criado em 6
Entrada na UTI em 6 de 2
Paciente 3 criado em 8
Entrada na UTI em 8 de 3
Paciente 4 criado em 10
Entrada na UTI em 10 de 4
Saída da UTI em 12 de 0
Paciente 5 criado em 12
Entrada na UTI em 12 de 5
Saída da UTI em 14 de 1
Paciente 6 criado em 14
Entrada na UTI em 14 de 6
Saída da UTI em 16 de 2
Paciente 7 criado em 16
Entrada na UTI em 16 de 7
Saída da UTI em 18 de 3
Paciente 8 criado em 18
Entrada na UTI em 18 de 8
Saída da UTI em 20 de 4
Paciente 9 criado em 20
Entrada na UTI em 20 de 9
Saída da UTI em 22 de 5
Paciente 10 criado em 22
Entrada na UTI em 22 de 10
Saída da UTI em 24 de 6
Paciente 11 criado em 24
Entrada na UTI em 24 de 11
Saída da UTI em 26 de 7
Paciente 12 criado em 26
Entrada na UTI em 26 de 12
Saída da UTI em 28 de 8
Paciente 13 criado em 28
Entrada na UTI em 28 de 13


In [11]:
import simpy
import random

class icusim():
    def __init__(self, env) -> None:
        self.env = env
        self.pacientes_solicitacao = 0
        self.pacientes_internados = 0
        self.pacientes_atendidos = 0
        self.leitos = simpy.Resource(env, capacity=2)

    def paciente(self, nome, tempo_internacao):
        print(f'Paciente {nome} solicitando UTI em {self.env.now}')
        leito_ocupado = self.leitos.request()
        yield leito_ocupado
        print(f'Entrada na UTI em {self.env.now} de {nome}' )
        self.pacientes_internados += 1
        yield self.env.timeout(tempo_internacao)
        print(f'Saída da UTI em {self.env.now} de {nome}')
        self.leitos.release(leito_ocupado)
        self.pacientes_atendidos += 1
        self.pacientes_internados -= 1

    def cria_paciente(self):
        paciente_nome = 0
        tempo_internacao = 10
        while True:
            yield env.timeout(random.randint(1, 5))
            self.pacientes_solicitacao += 1
            print(f'Paciente {paciente_nome} criado em {env.now}')
            env.process(self.paciente(paciente_nome, tempo_internacao))
            paciente_nome += 1
            
            
env = simpy.Environment()
hospital = icusim(env)
env.process(hospital.cria_paciente())
env.run(until=30)

print(f'Pacientes solicitados: {hospital.pacientes_solicitacao}')
print(f'Pacientes internados: {hospital.pacientes_internados}')
print(f'Pacientes atendidos: {hospital.pacientes_atendidos}')

Paciente 0 criado em 4
Paciente 0 solicitando UTI em 4
Entrada na UTI em 4 de 0
Paciente 1 criado em 8
Paciente 1 solicitando UTI em 8
Entrada na UTI em 8 de 1
Paciente 2 criado em 10
Paciente 2 solicitando UTI em 10
Paciente 3 criado em 13
Paciente 3 solicitando UTI em 13
Saída da UTI em 14 de 0
Entrada na UTI em 14 de 2
Paciente 4 criado em 15
Paciente 4 solicitando UTI em 15
Saída da UTI em 18 de 1
Entrada na UTI em 18 de 3
Paciente 5 criado em 20
Paciente 5 solicitando UTI em 20
Paciente 6 criado em 22
Paciente 6 solicitando UTI em 22
Saída da UTI em 24 de 2
Paciente 7 criado em 24
Paciente 7 solicitando UTI em 24
Entrada na UTI em 24 de 4
Paciente 8 criado em 26
Paciente 8 solicitando UTI em 26
Saída da UTI em 28 de 3
Entrada na UTI em 28 de 5
Pacientes solicitados: 9
Pacientes internados: 2
Pacientes atendidos: 4


In [27]:
# versão 2 
import simpy
import random
import numpy as np

class icusim():
    def __init__(self, env) -> None:
        self.env = env
        self.pacientes_solicitacao = 0
        self.pacientes_internados = 0
        self.pacientes_atendidos = 0
        
        self.leitos = simpy.Resource(env, capacity=10)

    def paciente(self, nome, tempo_internacao):
        print(f'Paciente {nome} solicitando UTI em {self.env.now}')
        leito_ocupado = self.leitos.request()
        yield leito_ocupado
        print(f'Entrada na UTI em {self.env.now} de {nome}' )
        self.pacientes_internados += 1
        yield self.env.timeout(tempo_internacao)
        print(f'Saída da UTI em {self.env.now} de {nome}')
        self.leitos.release(leito_ocupado)
        self.pacientes_atendidos += 1
        self.pacientes_internados -= 1

    def cria_paciente(self):
        paciente_nome = 0
        tempo_internacao = 7 * 24 # 10 dias
        while True:
            yield env.timeout(random.randint(2, 12))
            self.pacientes_solicitacao += 1
            print(f'Paciente {paciente_nome} criado em {env.now}')
            env.process(self.paciente(paciente_nome, tempo_internacao))
            paciente_nome += 1

# unidade temporal é HORAS

            
env = simpy.Environment()
hospital = icusim(env)
env.process(hospital.cria_paciente())
env.run(until=30*24)

print(f'Pacientes solicitados: {hospital.pacientes_solicitacao}')
print(f'Pacientes internados: {hospital.pacientes_internados}')
print(f'Pacientes atendidos: {hospital.pacientes_atendidos}')

Paciente 0 criado em 6
Paciente 0 solicitando UTI em 6
Entrada na UTI em 6 de 0
Paciente 1 criado em 17
Paciente 1 solicitando UTI em 17
Entrada na UTI em 17 de 1
Paciente 2 criado em 25
Paciente 2 solicitando UTI em 25
Entrada na UTI em 25 de 2
Paciente 3 criado em 29
Paciente 3 solicitando UTI em 29
Entrada na UTI em 29 de 3
Paciente 4 criado em 32
Paciente 4 solicitando UTI em 32
Entrada na UTI em 32 de 4
Paciente 5 criado em 38
Paciente 5 solicitando UTI em 38
Entrada na UTI em 38 de 5
Paciente 6 criado em 50
Paciente 6 solicitando UTI em 50
Entrada na UTI em 50 de 6
Paciente 7 criado em 56
Paciente 7 solicitando UTI em 56
Entrada na UTI em 56 de 7
Paciente 8 criado em 60
Paciente 8 solicitando UTI em 60
Entrada na UTI em 60 de 8
Paciente 9 criado em 65
Paciente 9 solicitando UTI em 65
Entrada na UTI em 65 de 9
Paciente 10 criado em 70
Paciente 10 solicitando UTI em 70
Paciente 11 criado em 81
Paciente 11 solicitando UTI em 81
Paciente 12 criado em 86
Paciente 12 solicitando UTI em

In [22]:
risco_obito_dia = 0.02
dias = 20
risco_obito = (1 + risco_obito_dia) ** dias - 1
print(risco_obito)

0.4859473959783549


In [26]:
# sorteia um número entre 0 e 1 em distribuição uniforme
#random.uniform(0, 1)

# sorteia um número entre 0 e 1 em distribuição normal
random.normalvariate(0, 1)

-0.8367876960291407

In [14]:
mean = 1
std = 0.5
lista = []
for i in range(1000):
    #lista.append(random.normalvariate(mean, std))
    lista.append(np.random.normal(mean, std))
lista

[1.0493044641528124,
 0.16222847959902997,
 1.8890669916279692,
 0.4723928079467099,
 1.3997870268631942,
 1.323191707205345,
 0.8958538806007648,
 0.9304514605619005,
 0.8407759555743892,
 0.5290496741127275,
 0.8829266652884893,
 1.8504040523545817,
 0.8854247735162359,
 1.0765566402528863,
 1.4838305497770625,
 0.920015045303181,
 0.9828906486429826,
 1.0051143531373765,
 0.9237634293243024,
 0.8880315457372624,
 1.4424981985638,
 1.4000267217900058,
 0.4447046226397676,
 0.2505150778742894,
 0.8050494232742784,
 1.0248129486786786,
 1.9412880085766129,
 0.5539195089887659,
 1.2219475206715797,
 -0.06752839627638973,
 1.1087935174593717,
 1.4052811933984746,
 1.2709160297439355,
 0.639912397998483,
 0.7635795948001234,
 0.5551700602089811,
 0.9628842238106038,
 0.5714358792148586,
 0.4791372454620112,
 1.5039321781153523,
 0.9612375543648918,
 1.0940947830661567,
 0.22978586187381056,
 0.3763218273140878,
 0.972311646572332,
 0.5775508870072752,
 1.738892147973506,
 1.62471881227696

In [15]:
def normalize(lst):
    min_val = min(lst)
    max_val = max(lst)
    range_val = max_val - min_val
    if range_val == 0:  # Avoid division by zero
        return [0]*len(lst)
    return [(num - min_val) / range_val for num in lst]

In [16]:
normalize(lista)

[0.48000816895480614,
 0.18842852197095714,
 0.7560360014912031,
 0.29037875728489265,
 0.5952109065878098,
 0.570034217493149,
 0.429569347748212,
 0.44094148534095484,
 0.4114653721977164,
 0.30900172682025717,
 0.42532020455995,
 0.7433275905083908,
 0.426141326456898,
 0.4889658907567308,
 0.622835801376086,
 0.43751106187114913,
 0.45817811468843167,
 0.46548299050964814,
 0.4387431462795533,
 0.4269981659353746,
 0.6092499613966192,
 0.5952896937079218,
 0.2812777203523207,
 0.21744810496807107,
 0.3997221508081041,
 0.47195786936334594,
 0.7732009185356911,
 0.31717637904844204,
 0.5367555077966715,
 0.11290801422824619,
 0.4995620712862453,
 0.5970168253216298,
 0.5528513336754717,
 0.345442025573849,
 0.3860911223984495,
 0.31758743209200857,
 0.45160205312030305,
 0.3229339659247587,
 0.29259563694803153,
 0.6294431560973699,
 0.4510607970114239,
 0.4947306342277403,
 0.21063446375929465,
 0.2588004673683384,
 0.4547008232918515,
 0.3249439536196834,
 0.7066739077573171,
 0.6